# Metoda CP
## Opis danych
Dane pochodzą z bazy o nazwie TUH Abnormal EEG Corpus (https://isip.piconepress.com/projects/tuh_eeg/downloads/tuh_eeg_abnormal/v2.0.0/). Są to zapisy sygnału EEG z jednego szpitala sklasyfikowane jako prawidłowe lub nieprawidłowe.

## Podstawowe informacje ułatwiające dalsze zrozumienie:
Do badania użyto standardowego układu elektrod 10-20, wygląda on następująco:

<p><IMG src="img/10-20.png" width=500></p>

Do każdego sygnału EEG jest załączony opis lekarza, który nie zawsze używa konkretnych nazw elektrod, tylko bardziej ogólne określenia.

* Parzyste numery elektrod są po prawej stronie, nieparzyste - po lewej
* Geneza literek w nazwach: pre-frontal (Fp), frontal (F), temporal (T), parietal (P), occipital (O), central (C)
* Zwolnienia w obszarach mózgu, K kompleksy - niskie częstotliwości
* Vertex, sharp waves - wyższe częstotliwości
* Wrzeciona snu (spindles) 12-16 Hz
    
## Drzewo decyzyjne
### Wybór cech
Lekarze oceniali sygnały EEG używając jedynie oczu i linijek - skupiali się więc na mocy danych częstotliwości w danych kanałach. Aby móc porównywać moje wyjaśnienia z ich opisami, wybrałam te same cechy. Dla każdego kanału obliczyłam moc sygnału w następujących pasmach częstotliwości:

* 0-2 Hz (delta)
* 1-3 Hz (delta)
* 2-4 Hz (delta)
* 3-6 Hz (theta)
* 4-8 Hz (theta)
* 6-10 Hz (alfa)
* 8-13 Hz (alfa)
* 10-15 Hz (beta)
* 13-18 Hz (beta)
* 15-21 Hz (beta)
* 18-24 Hz (beta)
* 21-27 Hz (beta)
* 24-30 Hz (gamma)
* 27-39 Hz (gamma)
* 30-49 Hz (gamma)
Uzyskałam w ten sposób 15x21=315 cech. Nie stosowałam żadnych redukcji cech, ponieważ chciałam uzyskać pełen obraz dla wszystkich cech.

Wyniki modelu na zbiorze ewaluacyjnym
* ACC - 0.84
* MCC - 0.67
* Spec - 0.87
* Sens - 0.85
### Wyjaśnianie konkretnych przypadków:
    
#### Przypadek pierwszy, klasyfikator poprawnie stwierdził nieprawidłowość EEG.
* Fragment opisu lekarza: "Abnormal EEG due to replacement of normal background primarily with a beta frequency pattern, superimposed asymmetry with relatively less beta and more suppression in the left particularly in the posterior quadrant"
* Rysunek przedstawiający wpływ zmienności wybranych cech:
    
<IMG src="img/sub_00007383_073.png">   
    
Akurat ta metoda wyjaśniania modelu dla patologicznych sygnałów EEG może być mało skuteczna, ponieważ nigdy w mózgu nie będzie sytuacji że tylko dla jednego kanału i jednego pasma sygnał będzie patologiczny. Jest to spowodowane tym że jednak mózg to sieć połączeń nerwowych gdzie występuje dużo skomplikowanych zależności między kanałami. Jeżeli więc dla patologicznego przypadku manipulujemy jedną wartością, to pozostałe patologiczne wartości pozostają nadal patologiczne i model nadal pokazuje że sygnał jest patologiczny.
    
#### Przypadek drugi, klasyfikator poprawnie stwierdził nieprawidłowość EEG.

* Rysunek przedstawiający wpływ zmienności wybranych cech których przebieg różni się od przypadku pierwszego: 
    
<IMG src="img/sub_00006531_065.png">   
    
Zmienności w przebiegach tych zależności nie da się jednoznacznie wytłumaczyć, ponieważ stan mózgu to całość na raz. Wystarczy więc że w jednym miejscu coś się zmieni, to inne kanały też powinny się dostosować do zmiany. W przypadku patologicznych zmian, te krzywe mogą być za każdym razem inne. 
    
#### Przypadek trzeci, klasyfikator poprawnie stwierdził prawidłowość EEG.

* Rysunek przedstawiający wpływ zmienności wybranych cech których przebieg różni się od przypadku pierwszego: 
    
<IMG src="img/sub_00004586_045.png">  
    
Jedynym przypadkiem gdzie można sprawdzić poprawność działania tej metody, jest analiza niepatologicznego EEG. Tutaj wyraźnie widać, że nasz przypadek jest w 'dołku' i jakiekolwiek odchylenia powodują wzrost prawdopodobieństwa że EEG jest patologiczne.

## Sieć konwolucyjna   
Jako drugi model wybrałam prostą sieć konwolucyjną, która na wejściu otrzymuje macierz tych samych danych co powyższy model, ale o kształcie (liczba_pasm x liczba_kanałów).

### Wyniki modelu na zbiorze ewaluacyjnym:
* ACC - 0.77
    
### Wyjaśnianie konkretnych przypadków (tych samych co dla Random Forest):
    
#### Przypadek pierwszy, klasyfikator poprawnie stwierdził nieprawidłowość EEG.
* Fragment opisu lekarza: "Abnormal EEG due to replacement of normal background primarily with a beta frequency pattern, superimposed asymmetry with relatively less beta and more suppression in the left particularly in the posterior quadrant"
* Rysunek przedstawiający wpływ zmienności wybranych cech:
    
<IMG src="img/sub_00007383_073_conv.png">  
    
#### Przypadek drugi, klasyfikator poprawnie stwierdził prawidłowość EEG.
* Rysunek przedstawiający wpływ zmienności wybranych cech:

<IMG src="img/sub_00004586_045_conv.png">     
    
Wydaje mi się że ta metoda nie sprawdziła się dla tego modelu. Wyniki są mniej jednoznaczne co znacznie utrudnia interpretacje. Może to wynikać z większej złożoności modelu i jednocześnie mniejszej wartości acc.
    
# Appendix 

In [1]:
import os
import pandas as pd
import time
from pathlib import Path
import numpy as np
import dalex as dx
import matplotlib.pyplot as plt
import torch
import torch.nn as nn
import torch.nn.functional as F
import copy
from einops import rearrange
from skimage import color

from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import cross_val_score
from sklearn.metrics import matthews_corrcoef, confusion_matrix, accuracy_score

/dmj/fizmed/mpoziomska/.pyenv/versions/3.6.10/envs/eeg_auto/lib/python3.6/site-packages/tqdm/auto.py:22: TqdmWarning:

IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html



In [2]:
ch_names = ['FP1', 'FP2', 'F3', 'F4', 'C3', 'C4', 'P3', 'P4', 'O1', 'O2', 'F7', 
            'F8', 'T3', 'T4', 'T5', 'T6', 'A1', 'A2', 'FZ', 'CZ', 'PZ']

BAND_LIMITS = np.array([[ 0,  2, 'delta'],
       [ 1,  3, 'delta'],
       [ 2,  4, 'delta'],
       [ 3,  6, 'theta'],
       [ 4,  8, 'theta'],
       [ 6, 10, 'alfa'],
       [ 8, 13, 'alfa'],
       [10, 15, 'beta'],
       [13, 18, 'beta'],
       [15, 21, 'beta'],
       [18, 24, 'beta'],
       [21, 27, 'beta'],
       [24, 30, 'gamma'],
       [27, 39, 'gamma'],
       [30, 49, 'gamma']])

In [3]:
X_train = pd.read_csv('data/X_train.csv')
Y_train = np.load('data/Y_train.npy')
X_eval = pd.read_csv('data/X_eval.csv')
Y_eval = np.load('data/Y_eval.npy')
Y_eval_all = np.load('data/Y_eval_all.npy', allow_pickle=True)

# Model Random Forest

In [46]:
clf_eval = RandomForestClassifier(n_estimators=1600, max_depth=90, max_features="sqrt", min_samples_split=2, random_state=4, 
                             criterion='entropy', n_jobs=20)
clf_eval.fit(X_train, Y_train)
preds = clf_eval.predict(X_eval)
probs = clf_eval.predict_proba(X_eval)
acc = accuracy_score(Y_eval, preds)
mcc = matthews_corrcoef(Y_eval, preds)
tn, fp, fn, tp = confusion_matrix(Y_eval, preds).ravel()
spec = tn / (tn+fp)
sens = tp / (tp + fp)
print(acc, mcc, spec, sens)

0.8369565217391305 0.671317113342619 0.8866666666666667 0.8521739130434782


In [50]:
def TP(pred, true):
    return (pred == 1) * (true == 1)

def FP(pred, true):
    return (pred == 1) * (true == 0)

def TN(pred, true):
    return (pred == 0) * (true == 0)

def FN(pred, true):
    return (pred == 0) * (true == 1)

d_idx = {4: [['A1', 30, 49, 'gamma'], ['A1', 2, 4, 'delta'], ['F8', 3, 6, 'theta'], ['F4', 1, 3, 'delta']], 10: [['A1', 30, 49, 'gamma'], ['A1', 2, 4, 'delta']], 12: [['C3', 30, 49, 'gamma'], ['C4', 2, 4, 'delta'], ['F8', 3, 6, 'theta'], ['F4', 1, 3, 'delta']]}
D = {'TP': (TP, [4, 10]), 'FP': (FP, []), 'TN': (TN, [12]), 'FN': (FN, [])}
sc = 1
res_path = f'img/'
clf_eval_exp = dx.Explainer(clf_eval, X_train, Y_train) 
for met, [fun, IDX] in D.items():
    sh = 1
    probs_met = probs[fun(preds, Y_eval)]
    Y_met = Y_eval[fun(preds, Y_eval)]
    Y_eval_all_met = Y_eval_all[fun(preds, Y_eval)]
    X_eval_met = X_eval.loc[fun(preds, Y_eval)].reset_index().drop('index', axis=1)
    preds_met = preds[fun(preds, Y_eval)]
    idx_sort = np.argsort(probs_met[:, sh])[::-1]
    
    for idx in IDX:
        
        prob = probs_met[idx]
        
        print(Y_eval_all_met[idx, 9])
        spli_path = Path(Y_eval_all_met[idx, 9]).parts
        with open(f'data/sub_{spli_path[6]}_{spli_path[5]}.txt', 'r') as f:
            desc = f.read()
        print(f"Case - {met}, Probability for 0: {prob[0]}, for 1: {prob[1]}")
        print(idx, preds_met[idx], Y_met[idx])
        print(desc)
        
        
        sick = X_eval_met.loc[idx]
        
        
        t0 = time.time()
        sub = clf_eval_exp.predict_profile(sick, processes=20)
        print(f'predict_profile {(time.time() - t0) / 60} min')
        
        variables = []
        for f in d_idx[idx]:
            ch, l, h, n = f
            name = f'ch_{ch}_{l}-{h} Hz ({n})'
            variables.append(name)
        
        t0 = time.time()
        fig = sub.plot(show=False, variables=variables)
        fig.write_image(res_path + f'sub_{spli_path[6]}_{spli_path[5]}.png')
        print(f'plot {(time.time() - t0) / 60} min')
        

Preparation of a new explainer is initiated

  -> data              : 2716 rows 315 cols
  -> target variable   : 2716 values
  -> model_class       : sklearn.ensemble.forest.RandomForestClassifier (default)
  -> label             : Not specified, model's class short name will be used. (default)
  -> predict function  : <function yhat_proba_default at 0x7fbba3f5d488> will be used (default)
  -> predict function  : Accepts pandas.DataFrame and numpy.ndarray.
  -> predicted values  : min = 0.00875, mean = 0.494, max = 1.0
  -> model type        : classification will be used (default)
  -> residual function : difference between y and yhat (default)
  -> residuals         : min = -0.37, mean = 0.00142, max = 0.358
  -> model_info        : package sklearn

A new explainer has been created!
data/pre/eval/abnormal/01_tcp_ar/073/00007383/s001_2010_02_25/00007383_s001_t001.h5
Case - TP, Probability for 0: 0.203125, for 1: 0.796875
4 1 1
CLINICAL HISTORY: n year old woman status post PEA code at

# Model sieci konwolucyjnej

In [51]:
def make_matrix(X):
    X_net = np.zeros((len(X), 1, len(BAND_LIMITS), len(ch_names)))
    for idx, sub in enumerate(X):
        for idx_ch, ch in enumerate(ch_names):
            for idx_b, [l, h, n] in enumerate(BAND_LIMITS):
                name = f'ch_{ch}_{l}-{h} Hz ({n})'
                X_net[idx, 0, idx_b, idx_ch] = sub[name == cols]
        
    return X_net.astype('float32')

cols = np.array(X_eval.columns)
X_eval_net = make_matrix(np.array(X_eval))
X_train_net = make_matrix(np.array(X_train))

In [52]:
class Shallow(nn.Module):
    
    def __init__(self, f1, f2):
        
        super().__init__()
        
        torch.manual_seed(seed)
        torch.cuda.manual_seed(seed)
        
        self.conv1 = nn.Conv2d(1, f1, (3, 4))
        
        self.conv2 = nn.Conv2d(f1, f2, (3, 4))
        
        self.avr = nn.AvgPool2d((3, 3), stride=(2, 2))
        
        self.lin = nn.Linear(int(350 * f2/10), 1)

    def forward(self, x):
        #print(x.shape)
        z = self.conv1(x)
        #print(z.shape)
        #print(z.shape)
        z = self.conv2(z)
        #print(z.shape)
        z = torch.square(z)
        #print(z.shape)
        z = self.avr(z)
        #print(z.shape)
        z = torch.log(z)
        z = rearrange(z, 'b c d e -> b (c d e)')
        z = self.lin(z)
        #print(z.shape)
        
        z = torch.sigmoid(z)
        
        return z

In [53]:
def evaluate(Net, X_eval, probs, MCC, Loss, epoch, trloss, t0):
        
    Net.eval()
    j = 0
    for i in range(len(X_eval_net) // batch_size):
        X = torch.from_numpy(X_eval_net[i * batch_size : min((i + 1) * batch_size, len(Y_eval) - 1)])

        X = X.to(device)

        nt = len(X)
        
        out = Net(X)
        probs[j:j+nt] = out.cpu().detach().numpy().copy()[:, 0]
        j += nt

    preds = get_pred(probs)
    acc = accuracy_score(Y_eval, preds)

    ACC[epoch-1] = acc
    Loss[epoch-1] = trloss
    print(f"Epoch {epoch}, time {(time.time() - t0) / 60:.2f} min, ACC {acc:.2f}, loss {trloss:.2f}")

def get_pred(probs):

    if np.sum(np.isnan(probs)) > 0:
        raise Warning("Nan values in probs!")
    else:
        return (probs.flatten() > 0.5) * 1

epochs = 16
batch_size = 64
probs = np.zeros((len(X_eval)))
ACC = np.zeros((epochs))
Loss = ACC.copy()
device = torch.device("cuda:0" if not torch.cuda.is_available() else "cpu")

LOSS = nn.BCELoss()
t0_mod = time.time()
seed = 123
torch.manual_seed(seed)
torch.cuda.manual_seed(seed)

def run_net(f1, f2):
    Net = Shallow(f1, f2)
    Net.to(device)
    optimizer = torch.optim.AdamW(Net.parameters())
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    for epoch in range(1, epochs + 1):
        t0_ep = time.time()

        Net.train()
        trloss = 0
        for i in range(len(X_train_net) // batch_size):
            X = torch.from_numpy(X_train_net[i * batch_size : min((i + 1) * batch_size, len(Y_train) - 1)])
            y = torch.from_numpy(np.array([Y_train[i * batch_size : min((i + 1) * batch_size, len(Y_train) - 1)]]).T.astype('float32'))

            X = X.to(device)
            y = y.to(device)

            optimizer.zero_grad()

            out = Net(X)

            loss = LOSS(out, y)

            loss.backward()
            optimizer.step()
            trloss += loss.data.item() * len(X)

        trloss /= len(X_train)

        evaluate(Net, X_eval, probs, ACC, Loss, epoch, trloss, t0_ep)

    print(f"Finished, time {(time.time() - t0_mod) / 60:.2f} min\n")
    return Net

Net = run_net(7, 12)
preds = get_pred(probs)

Epoch 1, time 0.00 min, ACC 0.71, loss 0.72
Epoch 2, time 0.00 min, ACC 0.72, loss 0.60
Epoch 3, time 0.00 min, ACC 0.74, loss 0.54
Epoch 4, time 0.00 min, ACC 0.73, loss 0.52
Epoch 5, time 0.00 min, ACC 0.73, loss 0.52
Epoch 6, time 0.00 min, ACC 0.74, loss 0.51
Epoch 7, time 0.00 min, ACC 0.75, loss 0.50
Epoch 8, time 0.00 min, ACC 0.75, loss 0.50
Epoch 9, time 0.00 min, ACC 0.75, loss 0.50
Epoch 10, time 0.00 min, ACC 0.75, loss 0.50
Epoch 11, time 0.00 min, ACC 0.75, loss 0.50
Epoch 12, time 0.00 min, ACC 0.76, loss 0.50
Epoch 13, time 0.00 min, ACC 0.76, loss 0.49
Epoch 14, time 0.00 min, ACC 0.76, loss 0.49
Epoch 15, time 0.00 min, ACC 0.76, loss 0.49
Epoch 16, time 0.00 min, ACC 0.77, loss 0.49
Finished, time 0.02 min



In [54]:
def TP(pred, true):
    return (pred == 1) * (true == 1)

def FP(pred, true):
    return (pred == 1) * (true == 0)

def TN(pred, true):
    return (pred == 0) * (true == 0)

def FN(pred, true):
    return (pred == 0) * (true == 1)


def predict(Net, X):
    if not isinstance(type(X), np.ndarray):
        X = np.array(X)
    X = make_matrix(X)
    if len(X.shape) == 3:
        X = X[:, np.newaxis]
    X = torch.from_numpy(X)
    
    preds = Net(X.to(device)).cpu().detach().numpy().copy()[:, 0]
    return preds

sc = 1
res_path = f'img/'
Net.eval()
sc=1
clf_eval_exp = dx.Explainer(Net, X_train, Y_train, predict_function = predict) 
subjects = ['data/pre/eval/abnormal/01_tcp_ar/073/00007383/s001_2010_02_25/00007383_s001_t001.h5', 'data/pre/eval/abnormal/01_tcp_ar/065/00006531/s002_2010_09_09/00006531_s002_t000.h5', 'data/pre/eval/normal/01_tcp_ar/045/00004586/s002_2009_10_07/00004586_s002_t001.h5']
d_idx = {0: [['A1', 30, 49, 'gamma'], ['A1', 2, 4, 'delta'], ['F8', 3, 6, 'theta'], ['F4', 1, 3, 'delta']], 1: [['A1', 30, 49, 'gamma'], ['A1', 2, 4, 'delta']], 2: [['C3', 30, 49, 'gamma'], ['C4', 2, 4, 'delta'], ['F8', 3, 6, 'theta'], ['F4', 1, 3, 'delta']]}
for i, s in enumerate(subjects):
    idx = np.argwhere(Y_eval_all[:, 9] == s)[0, 0]
    prob = probs[idx]

    spli_path = Path(Y_eval_all[idx, 9]).parts
    with open(f'data/sub_{spli_path[6]}_{spli_path[5]}.txt', 'r') as f:
        desc = f.read()
    print(f"Probability {prob}")
    print(idx, preds[idx], Y_eval[idx])
    print(desc)


    sick = np.array(X_eval)[idx].astype('float64')
    
    t0 = time.time()
    #print(sick.shape)
    sub = clf_eval_exp.predict_profile(sick)
    print(f'predict_profile {(time.time() - t0) / 60} min')

    variables = []
    for f in d_idx[i]:
        ch, l, h, n = f
        name = f'ch_{ch}_{l}-{h} Hz ({n})'
        variables.append(name)

    t0 = time.time()
    fig = sub.plot(show=False, variables=variables)
    fig.write_image(res_path + f'sub_{spli_path[6]}_{spli_path[5]}_conv.png')
    print(f'plot {(time.time() - t0) / 60} min')


Preparation of a new explainer is initiated

  -> data              : 2716 rows 315 cols
  -> target variable   : 2716 values
  -> model_class       : __main__.Shallow (default)
  -> label             : Not specified, model's class short name will be used. (default)
  -> predict function  : <function predict at 0x7fbbf23bf840> will be used
  -> predict function  : Accepts pandas.DataFrame and numpy.ndarray.
  -> predicted values  : min = 0.0218, mean = 0.49, max = 0.992
  -> model type        : 'model_type' not provided and cannot be extracted.
  -> model type        : Some functionalities won't be available.
  -> residual function : difference between y and yhat (default)
  -> residuals         : min = -0.949, mean = 0.00533, max = 0.956
  -> model_info        : package __main__

A new explainer has been created!
Probability 0.6664155125617981
72 1 1
CLINICAL HISTORY: n year old woman status post PEA code at 18:20 one day ago
with now facial twitching. History of heart failure, dement

Calculating ceteris paribus: 100%|██████████| 315/315 [01:28<00:00,  3.56it/s]


predict_profile 1.4854188323020936 min
plot 0.005863503615061442 min
Probability 0.9380353093147278
86 1 1
CLINICAL HISTORY: 48 year old woman with right MCA aneurysm, status post coil, craniotomy with EBD, and right sided edema.
MEDICATIONS: Dilantin, Ativan, Keppra, Neosynephrine
INTRODUCTION: Digital video EEG was performed at bedside using standard 10-20 system of electrode placement with 1 channel of EKG. The patient is intubated and poorly responsive.
DESCRIPTION OF THE RECORD: The background EEG demonstrates a continuous pattern. Both hemispheres are slow, but the left hemisphere is primarily a mixture of theta with delta. The right hemisphere demonstrates more significant arrhythmic delta activity with a right frontal breech rhythm and broadly contoured left frontal sharply contoured delta. In some sections of the record, particularly the right frontal temporal there is a dramatic attenuation of beta frequency activity. On 1 or 2 occasions as the patient drifts off into sleep v

Calculating ceteris paribus: 100%|██████████| 315/315 [01:28<00:00,  3.55it/s]


predict_profile 1.4875357151031494 min
plot 0.0038654605547587075 min
Probability 0.02401450090110302
16 0 0
CLINICAL HISTORY: 28 year old right handed male with seizures associated with hyperglycemia.
MEDICATIONS: None
INTRODUCTION: Digital video EEG was performed in lab using standard 10-20 system of electrode placement with 1 channel EKG. Hyperventilation and photic simulation are performed. This is an awake and drowsy record.
DESCRIPTION OF THE RECORD: In wakefulness, there is an 11 Hz, 50 microvolt posterior dominant rhythm with a small amount of low voltage, fronto-central beta activity. Brief drowsiness is characterized by anterior spread of the alpha rhythm. Hyperventilation produces an increase in amplitude of the background. Photic stimulation elicits a driving response.
HR: 66 bpm
IMPRESSION: Normal EEG.
CLINICAL CORRELATION: No focal or epileptiform features are observed in this EEG. This is the second normal EEG for this individual.


Calculating ceteris paribus: 100%|██████████| 315/315 [01:28<00:00,  3.55it/s]


predict_profile 1.4874565839767455 min
plot 0.006265882651011149 min
